<a href="https://colab.research.google.com/github/arnenoori/AP-CS/blob/main/Lab_6B_DATA_301_Arne_Noori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A code:
---

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [7]:
import requests
import pandas as pd

# getting list of Bay Wheels stations
response = requests.get('http://api.citybik.es/v2/networks/bay-wheels')
data = response.json()

bay_wheels_stations = []
for station in data['network']['stations']:
    bay_wheels_stations.append({
        'name': station['name'],
        'latitude': station['latitude'],
        'longitude': station['longitude']
    })

bay_wheels_stations_df = pd.DataFrame(bay_wheels_stations)

# getting address of each station
api_key = 'AsBlHnFSrqS73akLcr9zPTlBNWKVnmcYEP8GsY-e1uKyAaLWu9lm1XqK3OxZzjGX'
san_francisco_stations = 0
for i, row in bay_wheels_stations_df.iterrows():
    lat, lon = row['latitude'], row['longitude']
    url = f'http://dev.virtualearth.net/REST/v1/Locations/{lat},{lon}?key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'resourceSets' in data and len(data['resourceSets']) > 0 and 'resources' in data['resourceSets'][0] and len(data['resourceSets'][0]['resources']) > 0:
        address = data['resourceSets'][0]['resources'][0]['address']
        if address['locality'] == 'San Francisco':
            san_francisco_stations += 1
            print(f"Station {row['name']} is located in {address['locality']}, {address['adminDistrict']} County")

print(f"Number of Bay Wheels stations in San Francisco: {san_francisco_stations}")

Station Lane St at Revere Ave is located in San Francisco, CA County
Station Mendell St at Fairfax Ave is located in San Francisco, CA County
Station Williams Ave at 3rd St is located in San Francisco, CA County
Station Indiana St at Cesar Chavez St is located in San Francisco, CA County
Station Jennings St at Revere Ave is located in San Francisco, CA County
Station McKinnon Ave at 3rd St is located in San Francisco, CA County
Station Quesada Ave at 3rd St is located in San Francisco, CA County
Station Jerrold Ave at 3rd St is located in San Francisco, CA County
Station Oakdale Ave at Phelps St is located in San Francisco, CA County
Station Evans Ave at Mendell St is located in San Francisco, CA County
Station Williams Ave at Apollo St is located in San Francisco, CA County
Station Egbert Ave at Jennings St is located in San Francisco, CA County
Station Illinois St at Cesar Chavez St is located in San Francisco, CA County
Station Carroll Ave at 3rd St is located in San Francisco, CA C

**Explanation:** First, I read in the DataFrame of bike stations in the US from Part A and restricted it to stations in the Bay Wheels network. Using the Bing Maps Locations API, I obtained the address associated with each latitude and longitude coordinate for each station. I filtered for stations located in San Francisco and printed their names and locations. The output tells me that there are 15 Bay Wheels stations in San Francisco.

In [2]:
bay_wheels_stations_df

,name,latitude,longitude
0,Harmon St at Adeline St,37.849735,-122.270582
1,Fountain Alley at S 2nd St,37.336188,-121.889277
2,Oak St at 1st St,37.322125,-121.881090
3,Bestor Art Park,37.323678,-121.874119
4,5th St at Virginia St,37.325998,-121.877120
...,...,...,...
501,36th Ave at Kirkham St,37.758900,-122.495129
502,Irving St at 17th Ave,37.763732,-122.475220
503,Irving St at 18th Ave,37.763497,-122.476326
504,La Playa St at Lincoln Way,37.763874,-122.509526


## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [13]:
# Calculate driving distance between Coit Tower and each Bay Wheels station
coit_tower = (37.802747, -122.405861)

distances = []
for station in san_francisco_stations:
    loc = (station['latitude'], station['longitude'])
    url = f'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key={api_key}'
    data = {
        'origins': [{'latitude': coit_tower[0], 'longitude': coit_tower[1]}],
        'destinations': [{'latitude': loc[0], 'longitude': loc[1]}],
        'travelMode': 'driving'
    }
    response = requests.post(url, json=data)
    response_data = response.json()
    if 'resourceSets' in response_data and len(response_data['resourceSets']) > 0 and 'resources' in response_data['resourceSets'][0] and len(response_data['resourceSets'][0]['resources']) > 0:
        driving_distance = response_data['resourceSets'][0]['resources'][0]['results'][0]['travelDistance']
        distances.append({'name': station['name'], 'distance': driving_distance})

# find closest Bay Wheels station to Coit Tower
closest_station = min(distances, key=lambda x: x['distance'])
print(f"Nearest Bay Wheels bike station in San Francisco to Coit Tower: {closest_station['name']}")

Nearest Bay Wheels bike station in San Francisco to Coit Tower: Indiana St at Cesar Chavez St


**Answer is different from lab 6A...hmmm**

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.